# Setup

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')


In [23]:
def get_completion(prompt, schema, model="gpt-3.5-turbo"):
    messages = [
        {"role": "system", "content": "You are a helpful process documentation assistant."},
        {"role": "user", "content": prompt}
    ]
    functions = [
        {"name": "set_documentation", "parameters": schema}
    ]
    function_call = {"name": "set_documentation"}
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        functions=functions,
        function_call=function_call,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["function_call"]["arguments"]


# Read Template

In [32]:
import json

reader = open("../data/templates/documentation.schema.json")
template = json.load(reader)

In [28]:
print(template)

{'$schema': 'https://json-schema.org/draft/2020-12/schema', 'title': 'Process Documentation', 'description': 'A template for documenting a (business) process.', 'required': ['process', 'processResponsibility', 'input', 'supplierOfInput', 'processSteps', 'output', 'customerOfOutput', 'itSystems', 'resources'], 'type': 'object', 'properties': {'process': {'description': 'Your task is to generate a name for the given process.', 'type': 'string'}, 'processResponsibility': {'description': 'Your task is to describe the given process in maximal 3 sentences.', 'type': 'string'}, 'input': {'description': 'Your task is to list all input parameter in the given process.', 'type': 'array', 'items': {'type': 'string'}, 'uniqueItems': True}, 'supplierOfInput': {'description': 'Your task is to define from where the inputs come from.', 'type': 'array', 'items': {'type': 'string'}, 'uniqueItems': True}, 'processSteps': {'description': 'Your task is to list the process steps in their correct order.', 'ty

# Read BPMN Diagrams

## Application Process
<div>
    <img src="../images/application.png" width="900"/>
</div>


In [5]:
reader = open("../data/bpmn/application_process.bpmn")
application = reader.read()


## Cooking Process
<div>
    <img src="../images/cooking.png" width="900"/>
</div>


In [6]:
reader = open("../data/bpmn/cooking.bpmn")
cooking = reader.read()

## Invoice Process
<div>
    <img src="../images/invoice.png" width="900"/>
</div>


In [30]:
reader = open("../data/bpmn/invoice.bpmn")
invoice = reader.read()

# Prompts

In [24]:
prompt_application = f"""
Create a process documentation as a valid JSON object for the specified process delimited by triple backticks. \

Process: \
```{application}``` \
"""
response = get_completion(prompt_application, template, "gpt-4")
print(response)


{
  "process": "Recruitment Process",
  "processResponsibility": "This process is responsible for managing the recruitment of new employees, from the emergence of a vacancy to the selection of an applicant to fill the position.",
  "input": ["Vacancy details", "Job posting details", "Applications", "Exam results"],
  "supplierOfInput": ["Human Resources", "Hiring Manager", "Applicants", "Examiners"],
  "processSteps": ["Vacancy emerged", "Report vacancy", "Create job posting", "Application received", "Hold application exam", "Select applicant", "Position filled"],
  "output": ["Job posting", "Received applications", "Exam results", "Selected applicant"],
  "customerOfOutput": ["Job seekers", "Hiring Manager", "Examiners", "Human Resources"],
  "metrics": ["Time to fill vacancy", "Number of applications received", "Number of qualified applicants", "Time to hire"],
  "itSystems": ["HR Management System", "Job Posting Platform", "Applicant Tracking System"],
  "resources": ["HR personnel"

In [33]:
prompt_invoice = f"""
Create a process documentation as a valid JSON object for the specified process delimited by triple backticks. \

Process: \
```{invoice}``` \
"""
response = get_completion(prompt_invoice, template, "gpt-4")
print(response)

{
  "process": "Invoice Creation",
  "processResponsibility": "This process is responsible for creating an invoice when an item is sold, sending the invoice to the customer, and sending a reminder if the invoice is not settled. The process ends when the invoice is settled.",
  "input": ["Item sold", "invoice", "delivery", "invoiceId", "testInput"],
  "supplierOfInput": ["Sales Department", "Invoice Creation Process", "Delivery Department", "Invoice Creation Process", "Test System"],
  "processSteps": ["Item sold", "Create invoice", "Send invoice", "Received money?", "Send reminder", "Invoice settled"],
  "output": ["invoice"],
  "customerOfOutput": ["Customer"],
  "metrics": ["Number of invoices created", "Number of invoices sent", "Number of reminders sent", "Number of invoices settled"],
  "itSystems": ["Sales System", "Invoice System", "Reminder System"],
  "resources": ["Sales data", "Delivery data", "Invoice data"]
}


# More Complex Example


#### Order Process
<div>
    <img src="../images/order-process.png" width="900"/>
</div>


In [25]:
bpmn_without_bpmndi = """
<?xml version="1.0" encoding="UTF-8"?>
<bpmn:definitions xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL" xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI" xmlns:dc="http://www.omg.org/spec/DD/20100524/DC" xmlns:di="http://www.omg.org/spec/DD/20100524/DI" xmlns:camunda="http://camunda.org/schema/1.0/bpmn" id="Definitions_1" targetNamespace="http://bpmn.io/schema/bpmn" exporter="Camunda Modeler" exporterVersion="5.14.0">
  <bpmn:collaboration id="Collaboration_0e1vmb7">
    <bpmn:participant id="Participant_06ir2la" processRef="Process_1" />
    <bpmn:participant id="Participant_041ezgg" name="Accounting" />
    <bpmn:participant id="Participant_0v4qn1f" name="Inventory Management System" />
    <bpmn:messageFlow id="Flow_03c8sl3" sourceRef="Activity_0572g15" targetRef="Participant_041ezgg" />
    <bpmn:messageFlow id="Flow_0k3e93j" sourceRef="Activity_02i5xm0" targetRef="Participant_0v4qn1f" />
    <bpmn:messageFlow id="Flow_1wbn7he" sourceRef="Participant_0v4qn1f" targetRef="Activity_02i5xm0" />
  </bpmn:collaboration>
  <bpmn:process id="Process_1" isExecutable="false">
    <bpmn:startEvent id="StartEvent_1" name="Order received">
      <bpmn:outgoing>Flow_1aqzaf9</bpmn:outgoing>
    </bpmn:startEvent>
    <bpmn:userTask id="Activity_02i5xm0" name="Check availability">
      <bpmn:extensionElements>
        <camunda:inputOutput>
          <camunda:inputParameter name="order">${order}</camunda:inputParameter>
          <camunda:outputParameter name="isAvailable">${isAvailable}</camunda:outputParameter>
          <camunda:outputParameter name="storageYard">${storageYard}</camunda:outputParameter>
        </camunda:inputOutput>
      </bpmn:extensionElements>
      <bpmn:incoming>Flow_1aqzaf9</bpmn:incoming>
      <bpmn:outgoing>Flow_1b3thxg</bpmn:outgoing>
      <bpmn:property id="Property_0j6s4k3" name="__targetRef_placeholder" />
      <bpmn:dataInputAssociation id="DataInputAssociation_1oxr6rv">
        <bpmn:sourceRef>DataObjectReference_09rpu96</bpmn:sourceRef>
        <bpmn:targetRef>Property_0j6s4k3</bpmn:targetRef>
      </bpmn:dataInputAssociation>
    </bpmn:userTask>
    <bpmn:exclusiveGateway id="Gateway_0cjkzda" name="Products available?">
      <bpmn:incoming>Flow_1b3thxg</bpmn:incoming>
      <bpmn:outgoing>Flow_1erryay</bpmn:outgoing>
      <bpmn:outgoing>Flow_167v014</bpmn:outgoing>
    </bpmn:exclusiveGateway>
    <bpmn:userTask id="Activity_1nouf0v" name="Prepare order">
      <bpmn:extensionElements>
        <camunda:inputOutput>
          <camunda:inputParameter name="orderId">${order.prop("id")}</camunda:inputParameter>
          <camunda:inputParameter name="item">${order.prop("item")}</camunda:inputParameter>
          <camunda:inputParameter name="quantity">${order.prop("quantity")}</camunda:inputParameter>
          <camunda:inputParameter name="storageYard">${storageYard}</camunda:inputParameter>
          <camunda:outputParameter name="item">${item}</camunda:outputParameter>
          <camunda:outputParameter name="quantity">${quantity}</camunda:outputParameter>
        </camunda:inputOutput>
      </bpmn:extensionElements>
      <bpmn:incoming>Flow_1erryay</bpmn:incoming>
      <bpmn:outgoing>Flow_1iswizi</bpmn:outgoing>
    </bpmn:userTask>
    <bpmn:userTask id="Activity_0572g15" name="Deliver order">
      <bpmn:extensionElements>
        <camunda:inputOutput>
          <camunda:inputParameter name="invoiceAddress">${order.prop("invoiceAddress")}</camunda:inputParameter>
          <camunda:inputParameter name="deliveryAddress">${order.prop("deliveryAddress")}</camunda:inputParameter>
          <camunda:inputParameter name="item">${item}</camunda:inputParameter>
          <camunda:inputParameter name="quantity">${quantity}</camunda:inputParameter>
          <camunda:outputParameter name="deliveryBill">${deliveryBill}</camunda:outputParameter>
          <camunda:outputParameter name="invoice">${invoice}</camunda:outputParameter>
        </camunda:inputOutput>
      </bpmn:extensionElements>
      <bpmn:incoming>Flow_1iswizi</bpmn:incoming>
      <bpmn:incoming>Flow_1ti8aq3</bpmn:incoming>
      <bpmn:outgoing>Flow_0thm5o6</bpmn:outgoing>
      <bpmn:property id="Property_1dl5r6g" name="__targetRef_placeholder" />
      <bpmn:dataInputAssociation id="DataInputAssociation_1ioxhjd">
        <bpmn:sourceRef>DataObjectReference_0a4xslp</bpmn:sourceRef>
        <bpmn:targetRef>Property_1dl5r6g</bpmn:targetRef>
      </bpmn:dataInputAssociation>
      <bpmn:dataInputAssociation id="DataInputAssociation_17j0niu">
        <bpmn:sourceRef>DataObjectReference_0doca2i</bpmn:sourceRef>
        <bpmn:targetRef>Property_1dl5r6g</bpmn:targetRef>
      </bpmn:dataInputAssociation>
    </bpmn:userTask>
    <bpmn:exclusiveGateway id="Gateway_0enk64n" name="Delivery successful?" default="Flow_09bgqcq">
      <bpmn:incoming>Flow_0thm5o6</bpmn:incoming>
      <bpmn:outgoing>Flow_0cufsqq</bpmn:outgoing>
      <bpmn:outgoing>Flow_09bgqcq</bpmn:outgoing>
    </bpmn:exclusiveGateway>
    <bpmn:endEvent id="Event_0egkq7s" name="Order fullfilled">
      <bpmn:incoming>Flow_0cufsqq</bpmn:incoming>
    </bpmn:endEvent>
    <bpmn:serviceTask id="Activity_0bpppb5" name="Send cancellation" camunda:type="external" camunda:topic="sendCancellation">
      <bpmn:extensionElements>
        <camunda:inputOutput>
          <camunda:inputParameter name="order">${order}</camunda:inputParameter>
        </camunda:inputOutput>
      </bpmn:extensionElements>
      <bpmn:incoming>Flow_167v014</bpmn:incoming>
      <bpmn:outgoing>Flow_0vt19p8</bpmn:outgoing>
    </bpmn:serviceTask>
    <bpmn:endEvent id="Event_1xeeqax" name="Cancellation send">
      <bpmn:incoming>Flow_0vt19p8</bpmn:incoming>
    </bpmn:endEvent>
    <bpmn:endEvent id="Event_102l8sj" name="Order cancelled">
      <bpmn:incoming>Flow_0187d8h</bpmn:incoming>
      <bpmn:compensateEventDefinition id="CompensateEventDefinition_1nc7c2v" waitForCompletion="false" />
    </bpmn:endEvent>
    <bpmn:intermediateCatchEvent id="Event_0jsahpx" name="Next day">
      <bpmn:incoming>Flow_09bgqcq</bpmn:incoming>
      <bpmn:outgoing>Flow_1ti8aq3</bpmn:outgoing>
      <bpmn:timerEventDefinition id="TimerEventDefinition_1c0y8ik">
        <bpmn:timeDuration xsi:type="bpmn:tFormalExpression">P1D</bpmn:timeDuration>
      </bpmn:timerEventDefinition>
    </bpmn:intermediateCatchEvent>
    <bpmn:userTask id="Activity_0t4x0g8" name="Cancel order" isForCompensation="true">
      <bpmn:extensionElements />
    </bpmn:userTask>
    <bpmn:boundaryEvent id="Event_1nwp297" attachedToRef="Activity_1nouf0v">
      <bpmn:compensateEventDefinition id="CompensateEventDefinition_1mexgpq" />
    </bpmn:boundaryEvent>
    <bpmn:boundaryEvent id="Event_1fjysar" attachedToRef="Activity_0572g15">
      <bpmn:outgoing>Flow_0187d8h</bpmn:outgoing>
      <bpmn:errorEventDefinition id="ErrorEventDefinition_05qypjs" />
    </bpmn:boundaryEvent>
    <bpmn:sequenceFlow id="Flow_1aqzaf9" sourceRef="StartEvent_1" targetRef="Activity_02i5xm0" />
    <bpmn:sequenceFlow id="Flow_1b3thxg" sourceRef="Activity_02i5xm0" targetRef="Gateway_0cjkzda" />
    <bpmn:sequenceFlow id="Flow_1erryay" name="Yes" sourceRef="Gateway_0cjkzda" targetRef="Activity_1nouf0v">
      <bpmn:conditionExpression xsi:type="bpmn:tFormalExpression">${isAvailable == true}</bpmn:conditionExpression>
    </bpmn:sequenceFlow>
    <bpmn:sequenceFlow id="Flow_167v014" name="No" sourceRef="Gateway_0cjkzda" targetRef="Activity_0bpppb5">
      <bpmn:conditionExpression xsi:type="bpmn:tFormalExpression">${isAvailable == false}</bpmn:conditionExpression>
    </bpmn:sequenceFlow>
    <bpmn:sequenceFlow id="Flow_1iswizi" sourceRef="Activity_1nouf0v" targetRef="Activity_0572g15" />
    <bpmn:sequenceFlow id="Flow_1ti8aq3" sourceRef="Event_0jsahpx" targetRef="Activity_0572g15" />
    <bpmn:sequenceFlow id="Flow_0thm5o6" sourceRef="Activity_0572g15" targetRef="Gateway_0enk64n" />
    <bpmn:sequenceFlow id="Flow_0cufsqq" name="Yes" sourceRef="Gateway_0enk64n" targetRef="Event_0egkq7s" />
    <bpmn:sequenceFlow id="Flow_09bgqcq" name="No" sourceRef="Gateway_0enk64n" targetRef="Event_0jsahpx" />
    <bpmn:sequenceFlow id="Flow_0vt19p8" sourceRef="Activity_0bpppb5" targetRef="Event_1xeeqax" />
    <bpmn:sequenceFlow id="Flow_0187d8h" sourceRef="Event_1fjysar" targetRef="Event_102l8sj" />
    <bpmn:dataObjectReference id="DataObjectReference_09rpu96" name="Order" dataObjectRef="DataObject_0xmgeqb" />
    <bpmn:dataObject id="DataObject_0xmgeqb" />
    <bpmn:dataObjectReference id="DataObjectReference_0a4xslp" name="Delivery Bill" dataObjectRef="DataObject_14g4ktx" />
    <bpmn:dataObject id="DataObject_14g4ktx" />
    <bpmn:dataObjectReference id="DataObjectReference_0doca2i" name="Invoice" dataObjectRef="DataObject_0qk0tyi" />
    <bpmn:dataObject id="DataObject_0qk0tyi" />
    <bpmn:association id="Association_0ads7e6" associationDirection="One" sourceRef="Event_1nwp297" targetRef="Activity_0t4x0g8" />
  </bpmn:process>
</bpmn:definitions>
"""

In [29]:
prompt_order = f"""
Create a process documentation as a valid JSON object for the specified process delimited by triple backticks. \

Process: \
```{bpmn_without_bpmndi}``` \
"""
response = get_completion(prompt_order, template, "gpt-4")
print(response)


{
  "process": "Order Fulfillment",
  "processResponsibility": "This process is responsible for handling received orders, checking product availability, preparing and delivering the order. If the product is not available or delivery is unsuccessful, the order is cancelled.",
  "input": ["order"],
  "supplierOfInput": ["Customer"],
  "processSteps": ["Order received", "Check availability", "Prepare order", "Deliver order", "Order fullfilled or Cancelled"],
  "output": ["isAvailable", "storageYard", "item", "quantity", "deliveryBill", "invoice"],
  "customerOfOutput": ["Inventory Management System", "Delivery Service", "Accounting"],
  "metrics": ["Order processing time", "Number of orders fulfilled", "Number of orders cancelled", "Delivery success rate"],
  "itSystems": ["Inventory Management System", "Delivery Service System", "Accounting System"],
  "resources": ["Inventory", "Delivery personnel", "Accounting personnel"]
}
